## This script make the psi matrix into right shape.
This script also performs filtering to remove QC-failed samples, and remove exons with too many missing PSIs

In [3]:
library(ggstatsplot)
library(ggside)
library(dplyr)
library(summarytools)
library(ggfortify)

You can cite this package as:
     Patil, I. (2021). Visualizations with statistical details: The 'ggstatsplot' approach.
     Journal of Open Source Software, 6(61), 3167, doi:10.21105/joss.03167

Loading required package: ggplot2

Registered S3 method overwritten by 'ggside':
  method from   
  +.gg   ggplot2


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in system2("/usr/bin/otool", c("-L", shQuote(DSO)), stdout = TRUE):
“running command ''/usr/bin/otool' -L '/Users/suzheng/opt/anaconda3/envs/r4new/lib/R/library/tcltk/libs//tcltk.so'' had status 1”
system might not have X11 capabilities; in case of errors when using dfSummary(), set st_options(use.x11 = FALSE)



In [4]:
dir <- "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi"
setwd(dir)

In [5]:
len_psi <- read.csv("all.target_gene_psi.pasted.added_gene_symbols", header=T, row.names=1, sep="\t", check.names=F)
#len_psi <- read.csv("all.target_gene_inc.pasted.added_gene_symbols", header=T, row.names=1, sep="\t", check.names=F)

In [6]:
qc_failed_samples <- read.csv("all_qc_failed_samples.txt",  header=F)

In [7]:
len <- len_psi$length
psi <- len_psi[, -1]

In [8]:
qc_passed_samples <- !(colnames(psi) %in% qc_failed_samples[,1])

In [9]:
psi <- psi[,qc_passed_samples]

In [10]:
na_frac <- apply(psi, 1, function(x){sum(is.na(x))/length(x)})

### Distrubition of the missing value frequency for exons

In [16]:
psi_NA_Fraction_by_gene_df <- data.frame(na_frac = na_frac)
options(repr.plot.width=7.5, repr.plot.height=7.5)
# Generate histogram
psi_NA_Fraction_by_gene <- ggplot(psi_NA_Fraction_by_gene_df, aes(x=na_frac))
save(psi_NA_Fraction_by_gene, file=paste0(Sys.getenv("psi_fig_tables_RData_dir"), "/psi_NA_Fraction_by_gene.RData"))

In [10]:
psi <- psi[na_frac<0.4, ]

In [11]:
dim(psi)

[1] 19637  1420

In [12]:
gtex_meta <- read.csv("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/velocityRNA/data/meta_data/sample_attributes.txt", header=T, sep="\t")

In [13]:
gtex_meta %>% filter(SAMPID  %in% colnames(psi)) %>% dim()

[1] 1094   75

In [14]:
pw_meta <- read.csv("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/Broad_Perth_RNA_seq_titinopathy/meta/sample_ages_gender.txt", header=T, sep="\t")
gtex_meta <- read.csv("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/velocityRNA/data/meta_data/subject_phenotypes.txt", header=T, sep="\t")
rownames(pw_meta) <- pw_meta$sample
rownames(gtex_meta) <- gtex_meta$SUBJID

In [15]:
mcm_meta <- read.csv("/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/meta/E-MTAB-6814.sdrf.txt.clean.added_months", header=T, sep="\t")



In [16]:
gtex_sample_meta <- read.csv(
"/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2021/velocityRNA/data/meta_data/sample_attributes.txt",
    header=T,
    sep="\t"
)


In [17]:
gtex_sample_meta <- gtex_sample_meta[,c("SAMPID", "SMTS", "SMTSD")]
rownames(gtex_sample_meta) <- gtex_sample_meta$SAMPID

In [18]:
pw_index <- !grepl("GTEX", colnames(psi)) & !grepl("Human", colnames(psi))
gtex_index <- grepl("GTEX", colnames(psi))
mcm_index <- grepl("Human", colnames(psi))

In [19]:
pw_names <- intersect(colnames(psi)[pw_index], pw_meta$sample)
mcm_names <- intersect(colnames(psi)[mcm_index], mcm_meta$source.name)
psi_reordered <- cbind(psi[,c(pw_names,mcm_names)], psi[,gtex_index])
gtex_names <- colnames(psi[,gtex_index])

In [20]:
gtex_sample_meta_reordered <- gtex_sample_meta[gtex_names,]

In [21]:
row.names(mcm_meta) <- mcm_meta$source.name
pw_meta_reordered <- pw_meta[pw_names,]
mcm_meta_reordered <- mcm_meta[mcm_names,]

In [22]:
psi_gtex_subject_ids <- sapply(
    colnames(psi)[gtex_index],
    function(x)paste(strsplit(x, "-")[[1]][1], strsplit(x, "-")[[1]][2], sep="-")
    )
gtex_meta_reordered <- gtex_meta[match(psi_gtex_subject_ids, gtex_meta$SUBJID), ]


In [23]:
meta <- data.frame(month=c(pw_meta_reordered[, "month"], mcm_meta_reordered$month,12*gtex_meta_reordered[, "AGE"]),
                   gender=c(pw_meta_reordered[, "genderM1F2"], mcm_meta_reordered$sex, gtex_meta_reordered[, "SEX"]),
                   group=c(rep("Pathwest", length(pw_names)), rep("MCM", dim(mcm_meta_reordered)[1]), rep("GTEx", dim(gtex_meta_reordered)[1])),
                   tissue=c(rep("skeletal_muscle", length(pw_names)), mcm_meta_reordered$organism.part, gtex_sample_meta_reordered$SMTS),
                   subtissue=c(rep("skeletal_muscle", length(pw_names)), mcm_meta_reordered$organism.part, gtex_sample_meta_reordered$SMTSD),
                   disease=c(rep("normal", dim(psi_reordered)[2]))
                  )

meta$gender[meta$gender==1 | meta$gender=="male"] <- "Male"
meta$gender[meta$gender==2 | meta$gender=="female"] <- "Female"
rownames(meta) <- colnames(psi_reordered)
meta <- meta %>% mutate(age_group=ifelse(month < 1, "fetal_neonatal", ifelse(month < 18*12, "pediatric", "adult")))


In [24]:
meta$age_group <- factor(meta$age_group, levels=c("fetal_neonatal", "pediatric", "adult"))

In [25]:
meta[meta$tissue=="Heart", "tissue"] <- "heart"
meta[meta$tissue=="Muscle", "tissue"] <- "skeletal_muscle"
meta[meta$subtissue=="Muscle - Skeletal", "subtissue"] <- "skeletal_muscle"

Copied to overview_of_meta_data.html

In [27]:
save(psi, psi_reordered, meta, file = "/Users/suzheng/Documents/suzheng/UNSW/UNSWTasks/2022/psi_calculation/results/genomewide_PSI/psi_quanti/target_gene_psi/data/psi_meta.Rdata")

In [30]:
dim(psi)
dim(psi_reordered)
dim(meta)

[1] 19637  1420

[1] 19637  1416

[1] 1416    7

In [34]:
table(meta[c("tissue", "age_group")])

                 age_group
tissue            fetal_neonatal pediatric adult
  forebrain                   35        11     9
  heart                       39         8   457
  hindbrain                   38        12     9
  kidney                      32         8     0
  liver                       33         5     6
  ovary                       17         0     0
  skeletal_muscle              5        13   640
  testis                      27         7     5